In [1]:
import gymnasium as gym

env = gym.make("ALE/Seaquest-v5", render_mode="human")
observation, info = env.reset(seed=42)
for _ in range(1000):
   action = env.action_space.sample()  # this is where you would insert your policy
   observation, reward, terminated, truncated, info = env.step(action)

   if terminated or truncated:
      observation, info = env.reset()

env.close()

: 

In [1]:
import torch
import torch.nn.functional as F


# ensure the module is re-imported after changes
import importlib
import behavioral_cloning_bc.bc_model
importlib.reload(behavioral_cloning_bc.bc_model)
from behavioral_cloning_bc.bc_model import BCModel

def test_bc_model():
    # Define the dimensions
    state_dim = 10  # Example state dimension
    action_dim = 4  # Example action dimension
    global_seed = 42
    trial_idx = 1
    
    # Initialize the model
    model = BCModel(state_dim, action_dim, global_seed, trial_idx)
    
    # Create a dummy input tensor (batch size of 5)
    test_input = torch.randn(5, state_dim)
    
    # Forward pass
    output = model(test_input)
    
    # Check output
    print("Output shape:", output.shape)
    print("Output:", output)

    # Ensure the output shape is correct
    assert output.shape == (5, action_dim), "Output shape is incorrect"

    print("Test passed successfully!")

# Run the test function
test_bc_model()

Output shape: torch.Size([5, 4])
Output: tensor([[ 0.1545,  0.1581, -0.2320,  0.4472],
        [ 0.0414,  0.1270, -0.0323, -0.0104],
        [ 0.0392,  0.1510, -0.1316, -0.0186],
        [ 0.1197,  0.1317,  0.0128,  0.3574],
        [ 0.1148,  0.1007,  0.0682,  0.0845]], grad_fn=<AddmmBackward0>)
Test passed successfully!


In [6]:
import torch
import torch.nn.functional as F


# ensure the module is re-imported after changes
import importlib
import implicit_q_learning_iql.iql_model
importlib.reload(implicit_q_learning_iql.iql_model)
from implicit_q_learning_iql.iql_model import IQLModel

def test_iql_model():
    # Define the dimensions
    state_dim = 10  # Example state dimension
    action_dim = 4  # Example action dimension
    learning_rate=3e-4
    hidden_size=256
    tau=5e-3
    temperature=0.1
    expectile=0.7
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    global_seed = 42
    trial_idx = 1
    
    # Initialize the model
    model = IQLModel(state_dim, action_dim, learning_rate, hidden_size, tau, temperature, expectile, device, global_seed, trial_idx)
    
    # Create a dummy input tensor (batch size of 5)
    test_input = torch.randn(5, state_dim)
    
    # Forward pass
    output = model(test_input)
    
    # Check output
    print("Output shape:", output.shape)
    print("Output:", output)

    print("Test passed successfully!")

# Run the test function
test_iql_model()

NotImplementedError: Module [IQLModel] is missing the required "forward" function